In [ ]:
!pip install selenium

In [ ]:
import selenium
from selenium import webdriver
import time
import pandas as pd
import json
import os

## Extract Links

In [ ]:
def extract_links():
    # To extract links from given list of category urls and store in csv
    driver = webdriver.Chrome('/home/atul/scraping_driver/chromedriver')
    driver.maximize_window()
    time.sleep(5)
    comp_categ_list = []
    comp_list_link = []
    hyperlinks = ['https://www.alibaba.com/Medical-Devices_p100002906?spm=a27aq.13923566.9010494120.12.1d7231a8rQzV1u','https://www.alibaba.com/Medical-Consumables_p127866004?spm=a27aq.13923566.9010494120.13.1d7231a8rQzV1u','https://www.alibaba.com/Health-Care-Products_p100002908?spm=a27aq.13923566.9010494120.16.1d7231a8rQzV1u']
    i=0
    for link in hyperlinks:
        print(link)
        if i == 0:
            categ_str = 'Medical Devices'
        elif i == 1:
            categ_str = 'Medical Consumables'
        elif i == 2:
            categ_str = 'Health Care Products'
        driver.get(link)
        time.sleep(10)
        SCROLL_PAUSE_TIME = 0.7

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")+100

        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        time.sleep(5)
        elements = driver.find_elements_by_class_name('product-detail')
        print(len(elements))
        links = set()
        for element in elements:
            product_link = element.get_attribute('href')
            if product_link not in links:
                links.add(product_link)
            else:
                continue
        list_links = list(links)
        print("Category",i,len(list_links))
        comp_list_link.extend(list_links)
        comp_categ_list.extend([categ_str]*len(list_links))
        i+=1
    driver.quit()
    df = pd.DataFrame(zip(comp_categ_list,comp_list_link),columns = ['category','product_urls'])
    print(df)
    df.to_csv('alibaba_product_urls1.csv',index = None)

In [ ]:
extract_links()

In [ ]:
df = pd.read_csv('alibaba_product_urls1.csv') #update path of product_urls.csv

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
def get_company_overview(driver):
    keys = [i.text for i in driver.find_elements_by_css_selector('.field-title')]
    values = [i.get_attribute('title') if i.get_attribute('title')!='' else i.text for i in driver.find_elements_by_css_selector('.content-value') ]
    return dict(zip(keys,values))

In [ ]:
def get_production_capacity(driver):
    prod_cap_dic = {}
    prod_cap_dic['factory_info'] = {}
    keys = [i.text for i in driver.find_elements_by_css_selector('.icbu-shop-table-col-item .title div')]
    values = [i.text for i in driver.find_elements_by_css_selector('.icbu-shop-table-col-item .content span')]
    prod_cap_dic['factory_info'] = dict(zip(keys,values))
    return prod_cap_dic

In [ ]:
def get_rd_capacity(driver):
    rd_capacity = {}
    rd_capacity['certification'] = []
    keys = ['Picture','Certification Name','Issued By','Business Scope','Available Date','Verified']
    li = driver.find_elements_by_css_selector('.icbu-pc-cpRDCapacity .next-table-cell .next-table-cell-wrapper')
    val = []
    for element in li:
        if element.text=='':
            try:
                new_val = element.find_element_by_tag_name('img').get_attribute('src')
            except:
                new_val = element.text
        else:
            new_val = element.text
        val.append(new_val)
    i=0
    while i< len(val):
        d = dict(zip(keys,val[i:i+6]))
        i+=6
        rd_capacity['certification'].append(d)
    #rd_capacity['no_of_people'] = [i.text for i in driver.find_elements_by_css_selector('.icbu-pc-cpRDCapacity .infoList-mod-field~ .infoList-mod-field+ .infoList-mod-field .content div')][-1]
    return rd_capacity

In [ ]:
def get_trade_capacity(driver):
    tr_capacity = {}
    tr_capacity['main_markets and products'] = []
    keys = ['Main Markets', 'Total Revenue(%)','Main Product(s)','Verified']
    li = [i.text for i in driver.find_elements_by_css_selector('.icbu-pc-cpTradeCapability .next-table-cell .next-table-cell-wrapper')]
    i=0
    while i< len(li):
        d = dict(zip(keys,li[i:i+4]))
        i+=4
        tr_capacity['main_markets and products'].append(d)
    return tr_capacity

In [ ]:
def get_product_details(browser):
    product_details = {}
    try:
        title = browser.find_element_by_class_name("ma-title")
        product_details["title"] = title.text
        print("-------1-------")
    except Exception as e:
        print(e)

    try:
        icons = browser.find_element_by_class_name("util-clearfix")
        icon_list = icons.find_elements_by_css_selector("img")
        images_list = []
        for icon in icon_list:
#                 icon.click()
            images_list.append(str(icon.get_attribute("src")).replace("_50x50.jpg",""))
        product_details["product_images"] = images_list
        print("-------2-------")
    except Exception as e:
        print(e)

#     details = browser.find_element_by_class_name("module-productPackagingAndQuickDetail")
#     print(details.text)

    overview = browser.find_element_by_class_name("do-overview")
    entries = overview.find_elements_by_class_name("do-entry")
    overview_dict = {}
    for entry in entries:
        title = entry.find_element_by_class_name("do-entry-title")
        overview_list = entry.find_element_by_class_name("do-entry-list")
        entry_items = overview_list.find_elements_by_tag_name("dl")
        item_dict = {}
        for item in entry_items:
            try:
                item_list = str(item.text).splitlines()
                item_dict[item_list[0]] = item_list[1]
            except Exception as e:
                print("List",e)
                continue
            print("-------3-------")
        overview_dict[title.text] = item_dict
    product_details["overview"] = overview_dict
 ###############################################################################################################   
    
    product_desc = {}
    try:
        child_eles = browser.find_elements_by_xpath("//*[@id='J-rich-text-description']/div")
        for child_div in child_eles:
            data_section = child_div.get_attribute("data-section-title")
            print(data_section)
            if data_section =="Product Description":
                print(data_section)
                try:
                    tables = child_div.find_elements_by_tag_name('table')[1]
                    table1_dict = {}
                    tr_list = tables.find_elements_by_tag_name('tr')
                    for tr in tr_list:
                        td = tr.find_elements_by_tag_name('td')
                        table1_dict[td[0].text] = td[1].text
                        print("-------4-------",table1_dict)
                        product_desc = table1_dict
                except Exception as e:
                    # Not a table, handling for image
                    print(e)
                # For images:
                
            elif data_section=="User Manual":
                manual_img = []
                try:
                    user_manual = child_div
                    user_manual_img = user_manual.find_element_by_tag_name('img')
                    manual_img.append(user_manual_img.get_attribute('data-src'))
                    print("")
                    print("-------7-------")
                except Exception as e:
                    print("User Manual:",e)
            elif data_section=="Company Overview":
                try:
                    company_overview = child_div
                    company_overview_imgs = company_overview.find_elements_by_tag_name('img')
                    for img in company_overview_imgs:
                        manual_img.append(img.get_attribute('data-src'))
                        print("-------8-------")
                    product_details["user_manual"] = manual_img
                except Exception as e:
                    print("Company Overview:",e)
            elif data_section=="Our Partners":                
                try:
                    partner = child_div
                    partner_imgs = partner.find_elements_by_tag_name('img')
                    for img in partner_imgs:
                        product_details["partner"] = img.get_attribute('data-src')
                        print("-------9-------")
                except:
                    print("Our Partners:",e)
            elif data_section=="Our Services":                
                try:
                    our_service = {}
                    services = child_div

                    service_table = services.find_elements_by_tag_name("table")[1]
                    tr_list = service_table.find_elements_by_tag_name('tr')
                    service_dict = {}
                    for tr in tr_list:
                        td = tr.find_elements_by_tag_name('td')
                        service_dict[td[0].text] = td[1].text
                    our_service["description"] = service_dict

                    service_img = services.find_element_by_tag_name('img')
                    our_service["images"] = service_img.get_attribute('data-src')

                    product_details["our_services"] = our_service
                except Exception as e:
                    print("Our services:",e)
            elif data_section=="Certifications":
                try:
                    certificate = child_div
                    certificate_img = certificate.find_element_by_tag_name('img')
                    product_details["certificate"] = certificate_img.get_attribute('data-src')
                except Exception as e:
                    print("certificate:",e)
            elif data_section=="Packing & Delivery":                
                try:
                    pack_delv_dict = {}
                    pack_deliv = child_div
                    pack_deliv_imgs = pack_deliv.find_elements_by_tag_name('img')
                    pack_delv_dict["packing"] = pack_deliv_imgs[0].get_attribute('data-src')
                #         print('---paking----',packing)

                    pack_delv_dict["delivery"] = pack_deliv_imgs[1].get_attribute('data-src')
                #         print('--delivery---',delivery)
                    delivery_table = pack_deliv.find_elements_by_tag_name("table")[1]
                    tr_list = delivery_table.find_elements_by_tag_name('tr')
                    delivery_dict = {}
                    for tr in tr_list:
                        td = tr.find_elements_by_tag_name('td')
                        delivery_dict[td[0].text] = td[1].text
                    pack_delv_dict["description"] = delivery_dict
                    product_details["packing_delivery"] = pack_delv_dict
                except Exception as e:
                    print("packaging:",e)
            elif data_section=="FAQ":
                try:
                    faqs = child_div
                #     print(faqs.text)
                    faq_list = faqs.find_elements_by_tag_name('span')
                    #if len(faq_list)==1:
                        
                    faq_dict = {}
                    for faq in faq_list:
                        if ":" in faq.text:
                            faq_qa = str(faq.text).split(":")
                            faq_dict[faq_qa[0]] = faq_qa[1]
                    product_details["FAQ"] = faq_dict
                except Exception as e:
                    print("faq:",e)
        try:
            product_details['image_content']={}
            dic = {}
            for p in child_eles:
                for ele in p.find_elements_by_tag_name('img'):
                    src = ele.get_attribute('data-src')
                    if src:
                        dic[p.get_attribute('data-section-title')]='https:'+src
                product_details['image_content']=dic
        except Exception as e:
            print("image content",e)
        return product_details
    except:
        print("Product desc not found !")  
        return product_details                  

In [ ]:
def get_product_company_details(driver):
    dic = {}
    dic['product_details']  = {}
    dic['product_details'] = get_product_details(driver)
    # company
    try:
        driver.find_element_by_css_selector('.active+ .next-tabs-nav-appear-active .tab-name').click()
        dic['company_profile'] = {}
        dic['company_profile']['company_overview'] = dict()
        dic['company_profile']['production_capacity'] = dict()
        dic['company_profile']['r&d_capacity'] = dict()
        dic['company_profile']['trade_capacity'] = dict()
        
        dic['company_profile']['company_overview'] = get_company_overview(driver)
        dic['company_profile']['production_capacity'] = get_production_capacity(driver)
        try:
            driver.find_element_by_css_selector('.icbu-pc-cpRDCapacity .view-more').click()
            dic['company_profile']['r&d_capacity'] = get_rd_capacity(driver)
        except Exception as e:
        #J-ls-grid-desc > div.tab-body > div.tab-body-pane.ls-icon.ls-company.show > div.alisite > div:nth-child(5) > div > div > div > div.mod-content > div > div.mod-view-more > div
            print("R&D",e)
        try:
            driver.find_element_by_css_selector('.icbu-pc-cpTradeCapability .view-more').click()
            dic['company_profile']['trade_capacity'] = get_trade_capacity(driver)
        except Exception as e:
            print("Trade",e)
    except Exception as e:
        print("Company",e)
    return dic

In [ ]:
def get_json_for_url(url):
    # To extract content from a product url
    driver = webdriver.Chrome('/home/atul/scraping_driver/chromedriver')
    driver.maximize_window()
    time.sleep(5)
    driver.get(url)
    time.sleep(5)
    product_name = driver.find_element_by_class_name('ma-title').get_attribute('title')
    SCROLL_PAUSE_TIME = 0.7

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")+100

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(10)
    dic = get_product_company_details(driver)
    driver.quit()
    with open('crawler_output_files/result_'+product_name+'.json', 'w') as fp:
        json.dump(dic, fp)
    return product_name,dic

In [ ]:
def store_multiple_json_files(df,n=len(df)):
    beg = time.time()
    print(n)
    full_json = {}
    product_names = []
    product_urls = []
    file_names = []
    count = 0 
    i = 0
    while count<n and i<len(df):
        
        try:
            print(df['product_urls'][i])
            product_name,dic = get_json_for_url(df['product_urls'][i])
            product_names.append(product_name)
            product_urls.append(df['product_urls'][i])
            file_names.append('result_'+product_name+'.json')
            full_json[i]={}
            count+=1
        except Exception as e:
            print("Page not found",e)
            #full_json[i]='Not Found'
            dic = 'Page not found'
        full_json[i]=dic
        i+=1
    output_df = pd.DataFrame({'product_name':product_names,'product_url':product_urls,'file_name':file_names})
    print("Time taken for ",n,"urls:",(time.time()-beg),"seconds")
    return output_df,full_json

## Crawl Links

In [ ]:
begin = time.time()
output_df,full_json = store_multiple_json_files(df,1)

In [ ]:
full_json

In [ ]:
ldf = pd.DataFrame(full_json).T

In [ ]:
ldf.to_csv('file.csv',index = None)

In [ ]:
ldf['product_details'][0]

In [ ]:
ldf

In [ ]:
full_json[0]

In [ ]:
output_df.to_csv('final_df.csv',index = None)

In [ ]:
list_size = len(full_json.keys())
title_lst = [None]*list_size
product_img_lst = [None]*list_size
overview_lst = [None]*list_size
our_services = [None]*list_size
packaging_delivery = [None]*list_size
faq = [None]*list_size
image_content = [None]*list_size
company_overview = [None]*list_size
production_capacity = [None]*list_size
r_d_capacity = [None]*list_size
trade_capacity = [None]*list_size
user_manual = [None]*list_size
for key in full_json.keys():
    if full_json[key]!='Page not found':
        title_lst[key] = str(full_json[key]['product_details']['title'])
        product_img_lst[key] = str(full_json[key]['product_details']['product_images'])
        overview_lst[key] = str(full_json[key]['product_details']['overview'])
        if full_json[key]['product_details'].get('user_manual'):
            user_manual[key] = str(full_json[key]['product_details']['user_manual'])
        if full_json[key]['product_details'].get('our_services'):
            our_services[key] = str(full_json[key]['product_details']['our_services'])

        if full_json[key]['product_details']['overview'].get('Packaging & Delivery'):
            packaging_delivery[key] = str(full_json[key]['product_details']['overview']['Packaging & Delivery'])

        if full_json[key]['product_details'].get('FAQ'):   
            faq[key] = str(full_json[key]['product_details']['FAQ'])


        image_content[key] = str(full_json[key]['product_details']['image_content'])

        company_overview[key] = str(full_json[key]['company_profile']['company_overview'])
        production_capacity[key] = str(full_json[key]['company_profile']['production_capacity'])
        r_d_capacity[key] = str(full_json[key]['company_profile']['r&d_capacity'])
        trade_capacity[key] = str(full_json[key]['company_profile']['trade_capacity'])

In [ ]:
df_dic = {'product_title':title_lst,
 'product_images':product_img_lst,
 'overview':overview_lst,
 'user_manual':user_manual,
 'our_services':our_services,
 'packaging_delivery':packaging_delivery,
 'FAQ': faq,
 'image_content':image_content,
 'company_overview':company_overview,
 'production_capacity':production_capacity,
 'r&d_capacity':r_d_capacity,
 'trade_capacity':trade_capacity
}

In [ ]:
final_df1 = pd.DataFrame(df_dic)

In [ ]:
final_df1.isna().sum()

In [ ]:
final_df1['product_url'] = df['product_urls']

In [ ]:
final_df1['category'] = df['category']

In [ ]:
final_df1

In [ ]:
final_df1.to_csv('final_output_file.csv',index = None)

In [ ]:
df.head()

In [ ]:
final_df1.append(df,axis =1)

In [ ]:
with open('all_content.json','w') as f:
    json.dump(full_json,f)